# About this notebook

This notebook is a quick tutorial into the [Pandas](http://pandas.pydata.org/) library. It was written for the [2016 Construction grammar course](http://budling.hu/elmnyelv/index.php/Constructions2016) at the [Department of Theoretical Linguistics](http://www.nytud.hu/tlp/index.html). Its main aim is to provide easier access to the [Tádé korpusz](http://hlt.bme.hu/hu/resources/tade), on which most of the experiments in the course are based.

# How to start the notebook

(Fork and) Get the [repository from GitHub](https://github.com/DavidNemeskey/Tade-corpus-tools), enter into the notebooks directory, then run
```
jupyter notebook
```
You will see the list of all notebooks in the directory (and below). Just click on the one you want to load.

# Pandas tutorial

So this is the actual tutorial part. We shall work on a small table in the same format as the [Tádé korpusz](http://hlt.bme.hu/hu/resources/tade). The experiment skeleton loads the actual corpus.

The code below is how one usually initializes a notebook.

In [27]:
import pandas as pd
# So that plots work correctly
%matplotlib inline   

import matplotlib
import numpy as np
from IPython.display import display  # For display()

## A short introduction to Pandas concepts

### DataFrame

A `DataFrame` is a table that (in this case) stores the Tádé data. We can read a table from file (see the skeleton), but here we will create one manually and then display its contents. There are two ways to do it:

1. use the regular Python `print()` function; same output as in a terminal
1. the `display()` function will output nice formatted HTML; the same thing happens if you just specify the object you want to print as the **last command in the cell**

In [28]:
df = pd.DataFrame(columns=['verb', 'frame', 'frame_freq', 'verb_freq', 'freq_ratio'], data=list(zip(*[
  ['van', 'van', 'néz', 'néz', 'rá+néz', 'rá+néz'],
  ['@', 'NP<CAS<DAT>>', 'NP<CAS<ACC>>_NP<CAS<DAT>>', 'NP<CAS<SBL>>', 'NP<CAS<SBL>>', 'NP<CAS<INS>>'],
  [362298, 56905, 356, 2752, 216, 34],
  [908829] * 2 + [19461] * 2 + [586] * 2,
  [0.398643, 0.062614, 0.0182929962489, 0.141411027183, 0.368600682594, 0.0580204778157]
])))


print(df)
display(df)  # Last command; here just writing `df` would work, too

     verb                      frame  frame_freq  verb_freq  freq_ratio
0     van                          @      362298     908829    0.398643
1     van               NP<CAS<DAT>>       56905     908829    0.062614
2     néz  NP<CAS<ACC>>_NP<CAS<DAT>>         356      19461    0.018293
3     néz               NP<CAS<SBL>>        2752      19461    0.141411
4  rá+néz               NP<CAS<SBL>>         216        586    0.368601
5  rá+néz               NP<CAS<INS>>          34        586    0.058020


,verb,frame,frame_freq,verb_freq,freq_ratio
0,van,@,362298,908829,0.398643
1,van,NP<CAS<DAT>>,56905,908829,0.062614
2,néz,NP<CAS<ACC>>_NP<CAS<DAT>>,356,19461,0.018293
3,néz,NP<CAS<SBL>>,2752,19461,0.141411
4,rá+néz,NP<CAS<SBL>>,216,586,0.368601
5,rá+néz,NP<CAS<INS>>,34,586,0.058020


Now this frame is rather small, but the whole dataset has 1158484 rows. If we tried to print it, it might be too much for the browser; and even if not, it would certainly make our notebook useless. So let's just have a look at the first few lines! You can do it with the `head()` function (there is a `tail()` as well):

In [29]:
print(df.head(3), '\n')
print(df.tail(1))

  verb                      frame  frame_freq  verb_freq  freq_ratio
0  van                          @      362298     908829    0.398643
1  van               NP<CAS<DAT>>       56905     908829    0.062614
2  néz  NP<CAS<ACC>>_NP<CAS<DAT>>         356      19461    0.018293 

     verb         frame  frame_freq  verb_freq  freq_ratio
5  rá+néz  NP<CAS<INS>>          34        586     0.05802


***Note:*** As you can see, there is an index column in the table, which counts up from 0. It is possible to specify your own index; we will see examples for it later.

### Series

A row or column of a `DataFrame` is called a `Series`. You can select columns of a `DataFrame` with simple indexing:
1. `df['frame']` or `df.icol(1)` selects the second column and returns a `Series`
1. `df[['verb', 'frame']]` selects the first two and returns a two-column `DateFrame` (note the `[[]]`: we index with the _list_ (`['verb', 'frame']`))

In [30]:
print('A series:\n{}\n'.format(df['frame']))
print('A data frame:\n{}'.format(df[['verb' ,'frame']]))

A series:
0                            @
1                 NP<CAS<DAT>>
2    NP<CAS<ACC>>_NP<CAS<DAT>>
3                 NP<CAS<SBL>>
4                 NP<CAS<SBL>>
5                 NP<CAS<INS>>
Name: frame, dtype: object

A data frame:
     verb                      frame
0     van                          @
1     van               NP<CAS<DAT>>
2     néz  NP<CAS<ACC>>_NP<CAS<DAT>>
3     néz               NP<CAS<SBL>>
4  rá+néz               NP<CAS<SBL>>
5  rá+néz               NP<CAS<INS>>


Rows can be indexed by
1. `df.iloc[2]` or `df.irow(2)` selects the third row
1. `df.loc[2]` selects the row whose index is `2`; in this case, it is the same as the last one, but remember -- an index can be anything.

As with columns, you can select more rows, e.g. `df.iloc[[1, 2]]` returns a two-row `DataFrame`.

***Note:*** all the above indexing schemes work on `Series`.

In [31]:
print('A row:\n{}\n'.format(df.loc[2]))

A row:
verb                                néz
frame         NP<CAS<ACC>>_NP<CAS<DAT>>
frame_freq                          356
verb_freq                         19461
freq_ratio                     0.018293
Name: 2, dtype: object



### Filtering

There exists a data-driven way of selecting rows: filtering. We can set a condition for all rows, which will return a `Series` that tells us which rows satisfy the condition, and which do not:


In [32]:
# Selecting all rows where the frame count is at least 2000
print(df.frame_freq >= 2000)
# In order to actually select the rows, we just do; this will keep only the rows where the condition is True
df[df.frame_freq >= 2000]

0     True
1     True
2    False
3     True
4    False
5    False
Name: frame_freq, dtype: bool


,verb,frame,frame_freq,verb_freq,freq_ratio
0,van,@,362298,908829,0.398643
1,van,NP<CAS<DAT>>,56905,908829,0.062614
3,néz,NP<CAS<SBL>>,2752,19461,0.141411


### Grouping

In our table, we have 3 unique verbs. What if we want to aggregate all the rows that pertain to the same word (e.g. average `freq_ratio`, number of different frame configurations)? That is where grouping comes into the picture. The [`groupby()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby) function groups the rows by the specified column, and returns a [`GroupyBy`](http://pandas.pydata.org/pandas-docs/stable/api.html#groupby) object. The aggregation methods are defined on this object; below are the implementations of the two examples above.

In [33]:
groups = df.groupby(df.verb)
print('Number of frame configurations\n\n{}\n'.format(groups.count()))
print('Average of the the freq_ratio\n\n{}\n'.format(groups.mean()))

Number of frame configurations

        frame  frame_freq  verb_freq  freq_ratio
verb                                            
néz         2           2          2           2
rá+néz      2           2          2           2
van         2           2          2           2

Average of the the freq_ratio

        frame_freq  verb_freq  freq_ratio
verb                                     
néz           1554      19461    0.079852
rá+néz         125        586    0.213311
van         209601     908829    0.230629



We can spot two interesting things here.

**First**, the verb is not part of the columns anymore. This is because it is now the **index** of the new data frame. Now we can e.g. select rows by the verb with `loc()`: `gmeans.loc['néz']`. However, if you would like `verb` to be a regular column again, you can do that; see below:

In [34]:
indexed_df = df.set_index('verb')
print('We can specify a column (or a set thereof) as a the index with set_index()...:\n\n{}\n'.format(indexed_df))
print('... and we can convert the index back to a column with reset_index():\n\n{}'.format(indexed_df.reset_index()))

We can specify a column (or a set thereof) as a the index with set_index()...:

                            frame  frame_freq  verb_freq  freq_ratio
verb                                                                
van                             @      362298     908829    0.398643
van                  NP<CAS<DAT>>       56905     908829    0.062614
néz     NP<CAS<ACC>>_NP<CAS<DAT>>         356      19461    0.018293
néz                  NP<CAS<SBL>>        2752      19461    0.141411
rá+néz               NP<CAS<SBL>>         216        586    0.368601
rá+néz               NP<CAS<INS>>          34        586    0.058020

... and we can convert the index back to a column with reset_index():

     verb                      frame  frame_freq  verb_freq  freq_ratio
0     van                          @      362298     908829    0.398643
1     van               NP<CAS<DAT>>       56905     908829    0.062614
2     néz  NP<CAS<ACC>>_NP<CAS<DAT>>         356      19461    0.018293
3     né

**Second**, the `mean()` and `count()` functions ran on all columns, not just the one(s) we were interested in. In order to get only the mean of the `freq_ratio`, we just need to tell Pandas we only need that column. Both solution below will work (what is the difference?).

In [35]:
df[['verb', 'freq_ratio']].groupby('verb').mean()  # This is actually a DataFrame with a single column
df.groupby('verb').mean()['freq_ratio']            # This is a Series

verb
néz       0.079852
rá+néz    0.213311
van       0.230629
Name: freq_ratio, dtype: float64

## A few examples

To give you an idea of Pandas in action, this section lists a few examples, in which we extract various statistics from the data. Let's dig in!

### How many different verbs are in the corpus?

In other words, the number of unique elements in column one (`verb`). There are several ways to find it out:

1. Have Pandas [`describe()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.describe.html) the column for you. What it shows depends on the type of the column; for strings, the description will have a `unique` field. What do you think the other fields mean?
2. Just call the [`unique()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) function on the column. This returns an array of all the unique elements; all you need is to take the length of the array.
3. Or just do (2) in a single step with [`nunique()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html).

In [36]:
print('describe()-based solution:\n')    # \n means new line (i.e. Enter, Return, etc.)
desc = df.verb.describe()
print(desc, '\n')
print('Unique only: ', desc['unique'], '\n\n')

print('unique()-based solution:', len(df.verb.unique()), '\n\n')
print('nunique()-based solution:', df.verb.nunique())

describe()-based solution:

count          6
unique         3
top       rá+néz
freq           2
Name: verb, dtype: object 

Unique only:  3 


unique()-based solution: 3 


nunique()-based solution: 3


### In all, how many frames did we extract from our corpus?

Once again, there are different ways of doing this; for instance:

1. Sum the frame counts of all the different verb-frame pairs.
2. Notice that if we sum up the frame counts, we get the verb frequency. So we need only `sum()` the verb frequencies for each word. Now, this way is a bit longer:
  1. First, we do not need to work on the whole table, only the verbs and their counts; therefore, we extract these two columns to a new table.
  2. Then, we group this new table by the verbs; this gives us a [`Groupby`](http://pandas.pydata.org/pandas-docs/stable/api.html#groupby) object
  3. Since the verb frequencies are the same in all rows for a verb, we only need the `first()` row in each group
  4. Finally, we can `sum()` the filtered column...

In [37]:
print('Sum of frame counts: ', df.frame_freq.sum())

df_verb_and_freq = df[['verb', 'verb_freq']]
verb_and_freq_groups = df_verb_and_freq.groupby('verb')  # verb -> [verb_freq, verb_freq, verb_freq, ...]
verb_freqs = verb_and_freq_groups.first()                # verb -> verb_freq

print('Sum of verb counts: ', verb_freqs.verb_freq.sum())

# This works too: df[['verb', 'verb_freq']].groupby('verb').first().verb_freq.sum()

Sum of frame counts:  422561
Sum of verb counts:  928876


In [85]:
#pd.DataFrame(df.frame.str.split('_').tolist())

def split_frames(df):
    """
    Splits the frames into individual arguments and creates as many copies of the row
    as there are arguments in the frame. Makes certain calculations easier.
    """
    s = df.frame.str.split('_').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)  # Drops the second-level index
    s.name = 'frame'                 # Needs a name for join
    return df.drop('frame', axis=1).join(s)[df.columns]  # Keep the original column order

def normalize_frames(df, in_place=True):
    """
    This function does two things: first, it adds INF as a frame if the verb is a modal
    (i.e. the 'verb' column is of the format FiniteVerb_InfiniteVerb). Second, it replaces
    the '@' (empty) frame marker with numpy.nan for better grouping support later on.
    """
    df_ret = df if in_place else df.copy(deep=True)
    df_ret.frame = df_ret.frame.apply(lambda s: 'INF' if s == '@' else s + '_INF').where(
        df_ret.verb.str.contains('_', regex=False), other=df_ret.frame)
    df_ret.loc[df_ret.verb.str.contains('_', regex=False), 'verb'] = df_ret.verb.str.replace('_.*', '')
    df_ret.loc[df_ret.frame == '@', 'frame'] = np.nan
    return df_ret

def add_preverb_column(df):
    """Adds a preverb column to the data frame."""
    return df.assign(preverb=df.verb.str.replace('[+].+', '').where(df2.verb.str.contains('+', regex=False)))

display(df)
all_frames = df.frame_freq.sum()

df2 = df.append({'verb': 'meg+próbál_fut', 'frame': 'NP<CAS<ACC>>',
                 'frame_freq': 21, 'verb_freq': 56, 'freq_ratio': 0.375}, ignore_index=True)
df2 = df2.append({'verb': 'akar_fut', 'frame': '@',
                 'frame_freq': 5, 'verb_freq': 10, 'freq_ratio': 0.5}, ignore_index=True)
df2 = split_frames(normalize_frames(add_preverb_column(df2)))
display(df2)

display(df2[['frame', 'preverb']].rename(columns={'frame': 'count'}).groupby('preverb').count())
frame_counts = df2[['frame', 'verb']].rename(columns={'verb': 'count'}).groupby('frame').count()
preverb_counts = df2[['preverb', 'verb']].rename(columns={'verb': 'count'}).groupby('preverb').count()
display(frame_counts)
display(preverb_counts)

frame_sums = df2[['frame', 'frame_freq']].rename(columns={'frame_freq': 'C'}).groupby('frame').sum()
preverb_sums = df2[['preverb', 'frame_freq']].rename(columns={'frame_freq': 'B'}).groupby('preverb').sum()
cells = df2[['frame', 'preverb', 'frame_freq']].rename(columns={'frame_freq': 'A'}).groupby(['preverb', 'frame']).sum()
print(cells)
print(frame_sums)
print(preverb_sums)
display(cells)
display(preverb_sums)
display(frame_sums)

from scipy.stats import fisher_exact

full = cells.join(preverb_sums).join(frame_sums)
full['D'] = all_frames + full.A - full.B - full.C
full['Fisher'] = full.apply(lambda row: scipy.stats.fisher_exact([[row.A, row.B], [row.C, row.D]])[1], axis=1)
display(full)

#aha = pd.DataFrame({'apple': [0, 1, 2], 'banana': [3, 4, 5], 'cherry': ['good', 'bad', 'good']})
#aha['apple'][aha.cherry == 'bad'] = np.nan
#aha.loc[aha.cherry == 'bad', ['apple', 'banana']] = aha.loc[aha.cherry == 'bad', ['banana', 'apple']]
#aha.loc[aha.cherry == 'bad', 'apple'] = aha.banana + 1
#display(aha)
#aha.loc[aha.cherry == 'good', ['apple', 'banana']] = (aha[['banana','apple']] * 2).values
#aha.loc[aha.cherry == 'good', ['apple', 'banana']] = pd.concat([aha['banana'] * 3, aha['apple'] * 2], axis=1)
#display(aha)

,verb,frame,frame_freq,verb_freq,freq_ratio
0,van,@,362298,908829,0.398643
1,van,NP<CAS<DAT>>,56905,908829,0.062614
2,néz,NP<CAS<ACC>>_NP<CAS<DAT>>,356,19461,0.018293
3,néz,NP<CAS<SBL>>,2752,19461,0.141411
4,rá+néz,NP<CAS<SBL>>,216,586,0.368601
5,rá+néz,NP<CAS<INS>>,34,586,0.058020


,verb,frame,frame_freq,verb_freq,freq_ratio,preverb
0,van,NaN,362298,908829,0.398643,NaN
1,van,NP<CAS<DAT>>,56905,908829,0.062614,NaN
2,néz,NP<CAS<ACC>>,356,19461,0.018293,NaN
2,néz,NP<CAS<DAT>>,356,19461,0.018293,NaN
3,néz,NP<CAS<SBL>>,2752,19461,0.141411,NaN
4,rá+néz,NP<CAS<SBL>>,216,586,0.368601,rá
5,rá+néz,NP<CAS<INS>>,34,586,0.058020,rá
6,meg+próbál,NP<CAS<ACC>>,21,56,0.375000,meg
6,meg+próbál,INF,21,56,0.375000,meg
7,akar,INF,5,10,0.500000,NaN


,count
preverb,
meg,2
rá,2


,count
frame,
INF,2
NP<CAS<ACC>>,2
NP<CAS<DAT>>,2
NP<CAS<INS>>,1
NP<CAS<SBL>>,2


,count
preverb,
meg,2
rá,2


                        A
preverb frame            
meg     INF            21
        NP<CAS<ACC>>   21
rá      NP<CAS<INS>>   34
        NP<CAS<SBL>>  216
                  C
frame              
INF              26
NP<CAS<ACC>>    377
NP<CAS<DAT>>  57261
NP<CAS<INS>>     34
NP<CAS<SBL>>   2968
           B
preverb     
meg       42
rá       250


A
preverb frame            
meg     INF            21
        NP<CAS<ACC>>   21
rá      NP<CAS<INS>>   34
        NP<CAS<SBL>>  216

,B
preverb,
meg,42
rá,250


,C
frame,
INF,26
NP<CAS<ACC>>,377
NP<CAS<DAT>>,57261
NP<CAS<INS>>,34
NP<CAS<SBL>>,2968


A    B     C       D        Fisher
preverb frame                                             
meg     INF            21   42    26  422514  1.267137e-69
        NP<CAS<ACC>>   21   42   377  422163  4.422439e-48
rá      NP<CAS<INS>>   34  250    34  422311  4.815780e-90
        NP<CAS<SBL>>  216  250  2968  419559  0.000000e+00

In [77]:
for r in full.iterrows():
    print(r[1])
    break
    
#fisher_exact(full[['A','B']], full[['C', 'D']])
#[[full.A, full.B], [full.C, full.D]]
#full['YYY'] = full.apply(lambda row: [row['A'], row['B']], axis=1)
full['ZZZ'] = list([full.A, full.B])
full

A          21
B          42
C          26
D      422514
YYY        63
Name: (meg, INF), dtype: int64


ValueError: Length of values does not match length of index